## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pevek,RU,69.7008,170.3133,-15.20,98,100,3.96,overcast clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,-5.53,87,100,5.14,overcast clouds
2,2,Yellowknife,CA,62.4560,-114.3525,-14.58,82,86,5.01,overcast clouds
3,3,Butaritari,KI,3.0707,172.7902,82.54,72,41,17.25,scattered clouds
4,4,Chokurdakh,RU,70.6333,147.9167,-20.61,89,85,1.79,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? (F) "))
max_temp = float(input("What is the maximum temperature you would like for your trip? (F) "))

What is the minimum temperature you would like for your trip? (F) 75
What is the maximum temperature you would like for your trip? (F) 95


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp)
                                       & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Butaritari,KI,3.0707,172.7902,82.54,72,41,17.25,scattered clouds
7,7,Busselton,AU,-33.6500,115.3333,83.05,45,4,12.55,clear sky
8,8,Geraldton,AU,-28.7667,114.6000,84.61,58,0,18.41,clear sky
10,10,Bubaque,GW,11.2833,-15.8333,78.06,58,40,12.37,scattered clouds
16,16,Vaini,TO,-21.2000,-175.2000,81.90,76,99,11.95,light rain


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,82.54,scattered clouds,3.0707,172.7902,
7,Busselton,AU,83.05,clear sky,-33.6500,115.3333,
8,Geraldton,AU,84.61,clear sky,-28.7667,114.6000,
10,Bubaque,GW,78.06,scattered clouds,11.2833,-15.8333,
16,Vaini,TO,81.90,light rain,-21.2000,-175.2000,
20,Hobart,AU,83.01,clear sky,-42.8794,147.3294,
24,Rikitea,PF,76.84,overcast clouds,-23.1203,-134.9692,
26,Georgetown,MY,89.53,haze,5.4112,100.3354,
27,Tual,ID,82.89,overcast clouds,-5.6667,132.7500,
32,Atuona,PF,78.08,few clouds,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    city = row["City"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found; skipping {city}")

Hotel not found; skipping Thinadhoo
Hotel not found; skipping Bonfim
Hotel not found; skipping Edd
Hotel not found; skipping Poya
Hotel not found; skipping Gamba


In [26]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(inplace = True)
hotel_df.loc[(hotel_df["City"]=="Thinadhoo")]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [27]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,82.54,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
7,Busselton,AU,83.05,clear sky,-33.6500,115.3333,Observatory Guest House
8,Geraldton,AU,84.61,clear sky,-28.7667,114.6000,Broadwater Mariner Resort
10,Bubaque,GW,78.06,scattered clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
16,Vaini,TO,81.90,light rain,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
661,Bac Lieu,VN,84.04,scattered clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
663,Bowen,AU,88.02,clear sky,-20.0167,148.2333,Castle Motor Lodge Motel
668,Blackwater,AU,90.27,broken clouds,-23.5833,148.8833,Blackwater Motor Inn
671,Katherine,AU,88.12,light rain,-14.4667,132.2667,Pine Tree Motel


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))